# Fragility Comparison Analysis

This notebook generates comparison plots across different fragility levels:
- Success Rate vs Position Noise (2×2 grid)
- Break Rate vs Position Noise (2×2 grid with N/A for unbreakable)
- Gold highlight box around the reference fragility level

In [1]:
# ============================================================
# BLOCK 1: IMPORTS & LOCAL CONFIGURATION
# ============================================================

import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

# Import shared utilities
from analysis_utils import (
    # Constants
    ENTITY, PROJECT, COLORS,
    TAG_EVAL_PERFORMANCE, TAG_EVAL_NOISE, TAG_OLD_NOISE_EVAL,
    METRIC_SUCCESS, METRIC_BREAKS, METRIC_TOTAL,
    # Data functions
    get_best_checkpoint_per_run,
    download_eval_data,
    # Plotting functions
    plot_multi_panel_grid,
    print_data_summary,
)

# ============================================================
# LOCAL CONFIGURATION (specific to this analysis)
# ============================================================

# Fragility Levels - keys are internal names, each contains method tags
FRAGILITY_LEVELS = {
    #Z-Contact Included
    #"10N": {
    #    "Pose(1mm)": "pose_perf-comp:2026-01-05_10:54",
    #    "Pose(2.5mm)": "pose_25mm-noise:2026-01-19_07:50",
    #    "MATCH(1mm)": "LCLoP_fix_perf-comp:2026-01-09_11:17",
    #    "Hybrid-Basic(1mm)": "basic-hybrid_perf-comp:2026-01-05_10:54",
    #},
    #"15N": {
    #    "Pose(1mm)": "pose_task_frag:2026-01-06_00:52",
    #    "Pose(2.5mm)": "pose_25mm-noise:2026-01-19_07:50_15N",
    #    "MATCH(1mm)": "LCLoP_task_frag:2026-01-06_00:27",
    #    "Hybrid": "basic-hybrid_task_frag:2026-01-06_00:56",
    #},
    #"20N": {
    #    "Pose(1mm)": "pose_task_frag:2026-01-06_00:09",
    #    "Pose(2.5mm)": "pose_25mm-noise:2026-01-19_07:50_20N",
    #    "MATCH(1mm)": "LCLoP_task_frag:2026-01-06_00:09",
    #    "Hybrid": "basic-hybrid_task_frag:2026-01-06_09:18",
    #},
    #"10000N": {
    #    "Pose(1mm)": "pose_task_frag:2026-01-05_23:37",
    #    "Pose(2.5mm)": "pose_25mm-noise:2026-01-19_07:50_unbreak",
    #    "MATCH(1mm)": "LCLoP_task_frag:2026-01-05_22:12",
    #    "Hybrid": "basic-hybrid_task_frag:2026-01-06_00:08",
    #}, """


    #"10N": {
    #    "Pose(1mm)": "pose_breakable_f10N:2026-02-04_17:49",
    #    #"Pose(2.5mm)": "",
    #    "MATCH(1mm)": "MATCH_breakable_10N:2026-02-04_17:49",
    #    "Hybrid": "basic-hybrid_breakable_f10N:2026-02-04_17:49",
    #},
    #"15N": {
    #    "Pose(1mm)": "pose_breakable_f15N:2026-02-02_13:41",
    #    #"Pose(2.5mm)": "",
    #    "MATCH(1mm)": "MATCH_breakable_f15N:2026-02-02_13:41",
    #    "Hybrid": "basic-hybrid_breakable_f15N:2026-02-02_13:41",
    #},
    #"20N": {
    #    "Pose(1mm)": "pose_breakable_f20N:2026-02-02_13:41",
        #"Pose(2.5mm)": "",
    #    "MATCH(1mm)": "MATCH_breakable_f20N:2026-02-02_13:41",
    #    "Hybrid": "basic-hybrid_breakable_f20N:2026-02-02_13:41",
    #},
    #"10000N": {
    #    "Pose(1mm)": "pose_base-case:2026-02-02_13:40",
        #"Pose(2.5mm)": "pose_25mm:2026-02-02_13:40",
   #     "MATCH(1mm)": "MATCH_base-case:2026-02-04_17:46",
    #    "Hybrid": "basic-hybrid_base-case:2026-02-02_13:40",
    #}

    #"10N": {
    #    "Pose": "pose_squashed_breakable_f10:2026-02-21_10:59",
    #    "Hybrid-Basic": "hybrid_squashed_breakable_f10:2026-02-21_10:59",
    #    "Hybrid-MATCH": "MATCH_squashed_breakable_f10:2026-02-21_10:59",
    #},
    #"15N": {
    #    "Pose": "pose_squashed_breakable_f15:2026-02-22_12:12",
        #"Pose(2.5mm)": "",
    #    "Hybrid-Basic": "hybrid_squashed_breakable_f15:2026-02-22_12:12",
    #    "Hybrid-MATCH": "MATCH_squashed_breakable_f15:2026-02-22_12:12",
    #},
    #"20N": {
    #    "Pose": "pose_squashed_breakable_f20:2026-02-22_12:12",
    #    #"Pose(2.5mm)": "",
    #    "Hybrid-Basic": "hybrid_squashed_breakable_f20:2026-02-22_12:13",
    #    "Hybrid-MATCH": "MATCH_squashed_breakable_f20:2026-02-23_11:43",
    #},
    #"10000N": {
    #    "Pose": "pose_squashed_base-case_f10000:2026-02-22_12:12",
        #"Pose(2.5mm)": "",
    #    "Hybrid-Basic": "hybrid_squashed_base-case_f10000:2026-02-22_12:26",
    #    "Hybrid-MATCH": "MATCH_squashed_base-case_f10000:2026-02-22_12:12",
    #}

    "10N": {
        "Pose": "pose_DR_breakable_f10:2026-02-25_14:14",
        # "Pose(2.5mm)":"",
        "Hybrid-Basic":"hybrid_DR_breakable_f10:2026-02-25_14:14",
        "Hybrid-MATCH":"MATCH_DR_breakable_f10:2026-02-25_14:14",
    },
    "15N": {
        "Pose": "pose_DR_breakable_f15:2026-02-25_14:15",
        #"Pose(2.5mm)": "",
        "Hybrid-Basic": "hybrid_DR_breakable_f15:2026-02-25_14:30",
        "Hybrid-MATCH": "MATCH_DR_breakable_f15:2026-02-25_14:14",
    },
    "20N": {
        "Pose": "pose_DR_breakable_f20:2026-02-25_17:29",
        #"Pose(2.5mm)": "",
        "Hybrid-Basic": "hybrid_DR_breakable_f20:2026-02-25_17:29",
        "Hybrid-MATCH": "MATCH_DR_breakable_f20:2026-02-25_17:29",
    },
    "10000N": {
        "Pose": "pose_squashed_base-case_f10000:2026-02-26_01:41",
        #"Pose(2.5mm)": "",
        "Hybrid-Basic": "hybrid_squashed_base-case_f10000:2026-02-26_01:43",
        "Hybrid-MATCH": "MATCH_squashed_base-case_f10000:2026-02-26_01:42",
    }
}

# Display name mapping
FRAGILITY_DISPLAY_NAMES = {
    "5N": "$F_{th}$ = 5N",
    "10N": "$F_{th}$ = 10N",
    "15N": "$F_{th}$ = 15N",
    "20N": "$F_{th}$ = 20N",
    "10000N": "Unbreakable",
}

# Noise Level Mapping: display label -> metric range string
NOISE_LEVELS = {
    "1mm": "0mm-1mm",
    "2.5mm": "1mm-2.5mm",
    "5mm": "2.5mm-5mm",
    "7.5mm": "5mm-7.5mm",
}

# Old Noise Eval Filter: None, "include", or "exclude"
OLD_NOISE_EVAL_FILTER = "exclude"

# Policy Selection
TOP_N_POLICIES = None
MAX_CHECKPOINT = None  # Set to int to limit checkpoint search (e.g., 2000000 for first 2M steps)

# Highlight Configuration
HIGHLIGHT_FRAGILITY = "10N"  # Which fragility to highlight with gold box, or None

# N/A panels (where break rate is not applicable)
NA_FRAGILITIES = ["10000N"]

# Plot Configuration
SUCCESS_Y_LIM = (0, 100)
SUCCESS_Y_TICKS = [0, 20, 40, 60, 80, 100]
BREAK_Y_LIM = (0, 7)
BREAK_Y_TICKS = list(range(7))

# Error type: "ci" for 95% confidence interval, "binary_se" for binary standard error
ERROR_TYPE = "ci"

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.6.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/skand/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
# ============================================================
# BLOCK 2: DETERMINE BEST POLICY
# ============================================================

api = wandb.Api()
best_checkpoints = defaultdict(dict)  # best_checkpoints[fragility][method]

for fragility, method_tags in FRAGILITY_LEVELS.items():
    print(f"\n{'='*60}")
    print(f"Fragility: {fragility}")
    print(f"{'='*60}")
    for method_name, method_tag in method_tags.items():
        print(f"\n  {method_name} ({method_tag}):")
        best_checkpoints[fragility][method_name] = get_best_checkpoint_per_run(
            api, method_tag, max_checkpoint=MAX_CHECKPOINT
        )

In [ ]:
# ============================================================
# BLOCK 3: DOWNLOAD DATA
# ============================================================

print(f"Old Noise Eval Filter: {OLD_NOISE_EVAL_FILTER}")

noise_data = defaultdict(dict)  # noise_data[fragility][method]

for fragility, method_tags in FRAGILITY_LEVELS.items():
    print(f"\n{'='*60}")
    print(f"Downloading data for Fragility: {fragility}")
    print(f"{'='*60}")
    for method_name, method_tag in method_tags.items():
        print(f"\n  {method_name}...")
        noise_data[fragility][method_name] = download_eval_data(
            api=api,
            method_tag=method_tag,
            best_checkpoints=best_checkpoints[fragility][method_name],
            level_mapping=NOISE_LEVELS,
            prefix_template="Noise_Eval({level})_Core",
            level_col_name="noise_level",
            eval_tag=TAG_EVAL_NOISE,
            old_noise_filter=OLD_NOISE_EVAL_FILTER,
        )

# Print summary
print("\n" + "="*60)
print("DATA SUMMARY")
print("="*60)
for fragility in FRAGILITY_LEVELS.keys():
    print(f"\n{fragility}:")
    for method_name in FRAGILITY_LEVELS[fragility].keys():
        df = noise_data[fragility][method_name]
        if not df.empty:
            num_runs = df["run_name"].nunique()
            print(f"  {method_name}: {num_runs} runs")
        else:
            print(f"  {method_name}: No data")

In [ ]:

import importlib
import analysis_utils
importlib.reload(analysis_utils)

# Import shared utilities
from analysis_utils import (
    # Constants
    ENTITY, PROJECT, COLORS,
    TAG_EVAL_PERFORMANCE, TAG_EVAL_NOISE, TAG_OLD_NOISE_EVAL,
    METRIC_SUCCESS, METRIC_BREAKS, METRIC_TOTAL,
    # Data functions
    get_best_checkpoint_per_run,
    download_eval_data,
    # Plotting functions
    plot_multi_panel_grid,
    print_data_summary,
)

In [ ]:
# ============================================================
# BLOCK 4: SUCCESS RATE COMPARISON FIGURE
# ============================================================

# Get method names from first fragility level
method_names = list(FRAGILITY_LEVELS[list(FRAGILITY_LEVELS.keys())[0]].keys())

fig, axes = plot_multi_panel_grid(
    data=dict(noise_data),
    panel_keys=list(FRAGILITY_LEVELS.keys()),
    panel_display_names=FRAGILITY_DISPLAY_NAMES,
    method_names=method_names,
    level_labels=list(NOISE_LEVELS.keys()),
    level_col="noise_level",
    metric="success",
    n_cols=2,
    suptitle="Success Rate vs Hole Position Noise Across Fragility Levels",
    x_label="Hole Localization Noise",
    y_label="Success Rate (%)",
    y_lim=SUCCESS_Y_LIM,
    y_ticks=SUCCESS_Y_TICKS,
    error_type=ERROR_TYPE,
    highlight_panel=HIGHLIGHT_FRAGILITY,
    filter_top_n=TOP_N_POLICIES,
    best_checkpoints=dict(best_checkpoints),
)
plt.show()

In [ ]:
# ============================================================
# BLOCK 5: BREAK RATE COMPARISON FIGURE
# ============================================================

fig, axes = plot_multi_panel_grid(
    data=dict(noise_data),
    panel_keys=list(FRAGILITY_LEVELS.keys()),
    panel_display_names=FRAGILITY_DISPLAY_NAMES,
    method_names=method_names,
    level_labels=list(NOISE_LEVELS.keys()),
    level_col="noise_level",
    metric="breaks",
    n_cols=2,
    suptitle="Break Rate vs Position Noise Across Fragility Levels",
    x_label="Position Noise",
    y_label="Break Rate (%)",
    y_lim=BREAK_Y_LIM,
    y_ticks=BREAK_Y_TICKS,
    error_type=ERROR_TYPE,
    highlight_panel=HIGHLIGHT_FRAGILITY,
    na_panels=NA_FRAGILITIES,
    na_text="N/A\n(Peg cannot break)",
    filter_top_n=TOP_N_POLICIES,
    best_checkpoints=dict(best_checkpoints),
)
plt.show()

In [ ]:
# ============================================================
# BLOCK 6: SUCCESS RATE - NON-HIGHLIGHTED FRAGILITIES (1x3)
# ============================================================

# Exclude the highlighted fragility level
non_highlight_keys = [k for k in FRAGILITY_LEVELS.keys() if k != HIGHLIGHT_FRAGILITY]

# IEEE double-column width = 7.16 in; match single-column font sizing
DOUBLE_COL_CELL_SIZE = (10.5 / 3, 2.6)
TOP_N_POLICIES=None
fig, axes = plot_multi_panel_grid(
    data=dict(noise_data),
    panel_keys=non_highlight_keys,
    panel_display_names=FRAGILITY_DISPLAY_NAMES,
    method_names=method_names,
    level_labels=list(NOISE_LEVELS.keys()),
    level_col="noise_level",
    metric="success",
    n_cols=3,
    #suptitle="Success Rate vs Localization Noise Across Fragilities",
    x_label="Hole Localization Noise",
    y_label="Success Rate (%)",
    y_lim=SUCCESS_Y_LIM,
    y_ticks=SUCCESS_Y_TICKS,
    error_type=ERROR_TYPE,
    filter_top_n=TOP_N_POLICIES,
    best_checkpoints=dict(best_checkpoints),
    center_x_label=True,
    figsize_per_cell=DOUBLE_COL_CELL_SIZE,
    legend_panel=0,
    legend_position="below"
)
plt.show()

In [ ]:
# ============================================================
# BLOCK 7: BREAK RATE - NON-HIGHLIGHTED FRAGILITIES (1x3)
# ============================================================

fig, axes = plot_multi_panel_grid(
    data=dict(noise_data),
    panel_keys=non_highlight_keys,
    panel_display_names=FRAGILITY_DISPLAY_NAMES,
    method_names=method_names,
    level_labels=list(NOISE_LEVELS.keys()),
    level_col="noise_level",
    metric="breaks",
    n_cols=3,
    #suptitle="Break Rate vs Position Noise (Non-Reference Fragilities)",
    x_label="Position Noise",
    y_label="Break Rate (%)",
    y_lim=BREAK_Y_LIM,
    y_ticks=BREAK_Y_TICKS,
    error_type=ERROR_TYPE,
    na_panels=NA_FRAGILITIES,
    na_text="N/A\n(Peg cannot break)",
    filter_top_n=TOP_N_POLICIES,
    best_checkpoints=dict(best_checkpoints),
    center_x_label=True,
    figsize_per_cell=DOUBLE_COL_CELL_SIZE,
    legend_loc = "upper right"
)
plt.show()